In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
%cd '/content/drive/My Drive/Colab Notebooks/db'
!ls

/content/drive/My Drive/Colab Notebooks/db
covid_19_data.csv  Meteorite_Landings.csv  test.hdf   test.parquet  test.xlsx
london_merged.csv  poblacion.csv	   test.json  test.pkl	    text.csv


In [3]:
import pandas as pd
import numpy as np
pd.__version__

'1.3.5'

In [4]:
df_pob = pd.read_csv('poblacion.csv')
df_pob

,Country,year,pop
0,Afghanistan,2015,3.441360e+07
1,Albania,2015,2.880703e+06
2,Algeria,2015,3.972802e+07
3,American Samoa,2015,5.581200e+04
4,Andorra,2015,7.801100e+04
...,...,...,...
1035,Pre-demographic dividend,2018,9.194854e+08
1036,Small states,2018,4.057532e+07
1037,South Asia,2018,1.814389e+09
1038,South Asia (IDA & IBRD),2018,1.814389e+09


In [5]:
pd.options.display.float_format = '{:,.1f}'.format #Formato para decimales de un float

In [6]:
df_pob['year'] = pd.Categorical(df_pob['year'].apply(str)) #Se cambia a tipo categorico
df_pob.dtypes

Country      object
year       category
pop         float64
dtype: object

In [7]:
idx_filtro = df_pob['Country'].isin(['Aruba', 'Colombia']) #isin crea un vector de variables bool
#Colocando falso a los paises que no se incluyen en la lista
idx_filtro

0       False
1       False
2       False
3       False
4       False
        ...  
1035    False
1036    False
1037    False
1038    False
1039    False
Name: Country, Length: 1040, dtype: bool

In [8]:
df_sample = df_pob[idx_filtro] #Creacion de una muestra de los 2 paises seleccionados anteriormente
df_sample

,Country,year,pop
9,Aruba,2015,"104,341.0"
42,Colombia,2015,"47,520,667.0"
269,Aruba,2016,"104,872.0"
302,Colombia,2016,"48,171,392.0"
529,Aruba,2017,"105,366.0"
562,Colombia,2017,"48,901,066.0"
789,Aruba,2018,"105,845.0"
822,Colombia,2018,"49,648,685.0"


In [9]:
df_sample.set_index(['Country', 'year']) #Estas 2 columnas definiran los indices del dataFrame

,,pop
Country,year,
Aruba,2015,"104,341.0"
Colombia,2015,"47,520,667.0"
Aruba,2016,"104,872.0"
Colombia,2016,"48,171,392.0"
Aruba,2017,"105,366.0"
Colombia,2017,"48,901,066.0"
Aruba,2018,"105,845.0"
Colombia,2018,"49,648,685.0"


In [10]:
df_sample = df_sample.set_index(['Country', 'year']).sort_index()
#Esta una buena forma de estructurar los datos y obtener un mejor análisis
#Se agrupan por el pais y se ordenan por años
df_sample

pop
Country  year             
Aruba    2015    104,341.0
         2016    104,872.0
         2017    105,366.0
         2018    105,845.0
Colombia 2015 47,520,667.0
         2016 48,171,392.0
         2017 48,901,066.0
         2018 49,648,685.0

In [11]:
df_sample.loc['Colombia',:] #Seleccion de ciertos datos

,pop
year,
2015,"47,520,667.0"
2016,"48,171,392.0"
2017,"48,901,066.0"
2018,"49,648,685.0"


In [12]:
df_sample.loc['Colombia',:].loc['2016',:] #Reduce más la seleccion al anterior

pop   48,171,392.0
Name: 2016, dtype: float64

In [13]:
df_sample.xs(['Aruba']) #Otra forma de seleccionar datos como loc

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.
  """Entry point for launching an IPython kernel.


,pop
year,
2015,"104,341.0"
2016,"104,872.0"
2017,"105,366.0"
2018,"105,845.0"


In [14]:
df_sample.xs(['Aruba', '2018']) #Una forma mas rápida de agrupar datos comparado con la línea 35; solo para multiples indices

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing lists as key for xs is deprecated and will be removed in a future version. Pass key as a tuple instead.
  """Entry point for launching an IPython kernel.


pop   105,845.0
Name: (Aruba, 2018), dtype: float64

In [15]:
#Cuando se refiere al segundo indice, hay que indicarlo con "level"
#Este selecciona los datos de todos los paises que coincidan con el año 2018
df_sample.xs('2018', level='year')

,pop
Country,
Aruba,"105,845.0"
Colombia,"49,648,685.0"


In [16]:
df_countries = df_pob.set_index(['Country', 'year']).sort_index(ascending = [True, True]) #Se agrupan con sort en descendente y se agregan 2 indices, el país y el año
df_countries

pop
Country     year             
Afghanistan 2015 34,413,603.0
            2016 35,383,128.0
            2017 36,296,400.0
            2018 37,172,386.0
Albania     2015  2,880,703.0
...                       ...
Zambia      2018 17,351,822.0
Zimbabwe    2015 13,814,629.0
            2016 14,030,390.0
            2017 14,236,745.0
            2018 14,439,018.0

[1040 rows x 1 columns]

In [17]:
ids = pd.IndexSlice #Tomar partes del DF, este guarda una funcion
df_countries.loc[ids['Aruba':'Austria', '2015':'2017'],:].sort_index() #Se usa la funcion de ids para un rango de datos deseados
#Para que funcione esta línea, arriba el ascending debe estar ambas como True

pop
Country   year             
Aruba     2015    104,341.0
          2016    104,872.0
          2017    105,366.0
Australia 2015 23,815,995.0
          2016 24,190,907.0
          2017 24,601,860.0
Austria   2015  8,642,699.0
          2016  8,736,668.0
          2017  8,797,566.0

In [18]:
df_countries.index.get_level_values(0) #Me muestra los datos del primer indice

Index(['Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Albania',
       'Albania', 'Albania', 'Albania', 'Algeria', 'Algeria',
       ...
       'Yemen, Rep.', 'Yemen, Rep.', 'Zambia', 'Zambia', 'Zambia', 'Zambia',
       'Zimbabwe', 'Zimbabwe', 'Zimbabwe', 'Zimbabwe'],
      dtype='object', name='Country', length=1040)

In [19]:
df_countries.index.get_level_values(1) #Me muestra los datos del segundo indice

CategoricalIndex(['2015', '2016', '2017', '2018', '2015', '2016', '2017',
                  '2018', '2015', '2016',
                  ...
                  '2017', '2018', '2015', '2016', '2017', '2018', '2015',
                  '2016', '2017', '2018'],
                 categories=['2015', '2016', '2017', '2018'], ordered=False, dtype='category', name='year', length=1040)

In [20]:
df_countries['pop']['Colombia']['2018'] #Seleccion de datos de las columnas usando los indices

49648685.0

In [21]:
df_countries.sum(level='year') #Este suma la poblacion con respecto al indice año

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """Entry point for launching an IPython kernel.


,pop
year,
2015,"65,679,147,019.0"
2016,"66,487,930,677.0"
2017,"67,294,176,701.0"
2018,"68,087,886,692.0"


In [22]:
df_sample

pop
Country  year             
Aruba    2015    104,341.0
         2016    104,872.0
         2017    105,366.0
         2018    105,845.0
Colombia 2015 47,520,667.0
         2016 48,171,392.0
         2017 48,901,066.0
         2018 49,648,685.0

In [23]:
df_sample.unstack('year') #Transforma a los diferentes valores de años en columnas

pop                                       
year             2015         2016         2017         2018
Country                                                     
Aruba       104,341.0    104,872.0    105,366.0    105,845.0
Colombia 47,520,667.0 48,171,392.0 48,901,066.0 49,648,685.0

In [24]:
df_sample.unstack('Country') #Transforma a los diferentes valores de Country en columnas

pop             
Country     Aruba     Colombia
year                          
2015    104,341.0 47,520,667.0
2016    104,872.0 48,171,392.0
2017    105,366.0 48,901,066.0
2018    105,845.0 49,648,685.0

In [29]:
df = pd.DataFrame({
    'edad' :     [ 10, 9, 13, 14, 12, 11, 12],
    'cm' : [ 115, 110, 130, 155, 125, 120, 125],
    'pais' :    [ 'co', 'mx', 'co', 'mx', 'mx', 'ch', 'ch'],
    'genero' :  [ 'F', 'M', 'M', 'M', 'F', 'F', 'F'],
    'Q1' : [ 5, 10, 8, np.nan, 7, 8, 3],
    'Q2' : [ 7, 9, 9, 8, 8, 8, 9.]
}, index = ['Ana','Benito','Camilo','Daniel','Erika','Paola','Gabriela'])

In [30]:
print(df.iloc[[4],[2]])

      pais
Erika   mx


In [35]:
pd.Series([10,8,5,0]) - pd.Series([0,1,6,12])

0    10
1     7
2    -1
3   -12
dtype: int64

In [37]:
[10,8,5,0] + [0,1,6,12]

[10, 8, 5, 0, 0, 1, 6, 12]

In [38]:
print(df.query("(edad >= 12) & (cm < 130) & (Q1 > 5)")['Q2'])

Erika   8.0
Name: Q2, dtype: float64


In [39]:
pd.NA | False

<NA>

In [40]:
print(df.loc[['Ana'],['cm']])

      cm
Ana  115
